![figure](lab5/politecnico_h-01.png)

# **Eletrónica Programável**
#### Licenciatura em Engenharia Eletrotécnica e de Computadores

## Trabalho Laboratorial 5 - Máquinas de Estados Finitos com DataPath

-----------------------


## Introdução ##
Neste trabalho vamos implementar um conjunto de máquinas de estados com *datapath* usando uma descrição comportamental em VHDL. Este documento não fornece detalhes de utilização do Vivado. Sempre que tiver dúvidas a este nível, consulte o enunciado do Lab1.


### Objetivos ###

Neste trabalho popomos a implementação de três circuitos que implementam máquinas de estados com datapath:
* Circuito divisor.
* Conversor Binário/BCD.
* Frequencímetro digital para baixas frequências.


Nas instruções abaixo **{sources}** refere-se a `C:\Xilinx\EEC_EP\sources` e **{labs}** refere-se a `(C:\Xilinx\EEC_EP\labs)`


### Referencias ###

Pong P. Chu. 2008. FPGA Prototyping by VHDL Examples: Xilinx Spartan-3 Version. 


_______________________

## Passo 1 - Circuito Divisor ##

### Passo 1.1 ### 

Pela sua complexidade, o circuito divisor não pode ser sintetizado de forma automática. Nesta secção vamos implementar um algoritmo de divisão de números inteiros de 8 bits. Na figura abaixo mostra-se um exemplo de 4 bits. Ele pode ser resumido em 4 passos:

1. Duplique a largura do dividendo acrescentado zeros à esquerda e coloque o divisor do lado esquerdo do dividendo aumentado.

2. Considere apenas os 4 dígitos mais à esquerda do dividendo. Se estes bits corresponderem a um número maior ou igual que o número do divisor, subtraia o divisor do dividendo e coloque um ‘1’ no quociente. Caso contrário, mantenha os bits do dividendo e coloque um ‘0’ no quociente.

3. Acrescente um bit do dividendo ao resultado anterior.

4. Repita os pontos 2 e 3 até que todos os bits do dividendo tenham sido utilizados
 

<div>
<img src="lab5/fig1.png" width=900"/>
</div>
                                   
                                   
A figura mostra também um esboço dos módulos de ação necessários. Inicialmente o divisor está armazenado no registo **d** e o dividendo expandido está nos registos **rh** e **rl**. A cada iteração, estes registos são deslocados à esquerda numa posição. Os 4 bits mais à esquerda (rh) são então comparados e eventualmente subtraídos ao divisor. À medida que o registo do dividendo vai sendo deslocado para a esquerda vai libertando espaço de memória que pode ser utilizado para acomodar os bits do quociente. No final, o registo **rl** possui todos os bits do quociente, enquanto o registo **rh** armazena o resto da divisão.

O diagrama ASMD deste circuito consiste em quatro estados: **idle**, **op**, **last** e **done**. A maioria das operações (comparação, subtração e deslocamento) são realizadas no estado **op**. Uma vez que o resto da divisão não deve ser deslocado, foi criado um estado adicional (last) que difere do estado anterior (op) apenas pela falta do deslocamento. 



* Comece por descarregar os arquivos fonte fornecidos (Moodle) e salve-os em `{sources}/lab5`.


* Abra o Vivado a partir do icon no ambinete de trabalho ou: `Start > All Programs > Xilinx Design Tools > Vivado 2022.1`. 


* Selecione **Create Project** para começar o wizard. Na caixa de diálogo *Create A New Vivado Project*, selecione **Next**.


* Clique no botão *Procurar* do campo de localização do projeto do novo formulário de projeto, navegue até `c:/Xilinx/EEC_EP/labs` e clique em **Select**. Digite *lab5* no campo de nome do projeto. Certifique-se de que a caixa *Create Project Subdirectory* esteja marcada. Clique em **Next**.


* Selecione a opção **RTL Project** no formulário *Project Type* e clique em **Next**. Certifique-se de que a caixa *Do not specify sources at this time* está marcada.


* No formulário **Default Part**, selecione a opção *Boards* e escolha a placa **Basys3**. Clique em **Next** e depois em **Finish** para criar o projeto Vivado.


* No painel *Sources*, selecione **Add Sources** e siga os passos necessários (*Add or create design sources*) para adicionar o ficheiro **div.vhd**, que contém a descrição VHDL do divisor.


* Abra o ficheiro no Vivado, observe o código. 


* Desenhe o diagrama de fluxo de operações RT (ASMD) e verifique que está de acordo com o algoritmo, tal como foi descrito.


______

## Passo 2 - Conversor Binário/BCD ##

### Passo 2.1 ### 

Para poder visualizar um número binário num visor, é necessário recorrer a um conversor de binário para BCD (Binary Coded Decimal), geralmente implementado como um registo de deslocamentos. Na figura abaixo é ilustrado o processo de conversão do número 12710 de binário ("1111111") para BCD. O algoritmo pode ser descrito desta forma:

1. Para cada dígito BCD (4 bits) no registo de deslocamentos, verifique se o número é maior do que 410. Se for este o caso, adicione o número 310.

2. Desloque todo o registo à esquerda uma posição, introduzindo o MSB da sequência binária no LSB do registo.

3. Repita os passos 1 e 2 até que todos os bits do número binário tenham sido utilizados.


<div>
<img src="lab5/fig2.png" width="500"/>
</div>



* A descrição VHDL deste circuito é fornecida no ficheiro **bin2bcd.vhd**. Adicione-o ao projeto Vivado (*Add or create design sources*). Abra o ficheiro, observe o código e verifique que está de acordo com o algoritmo, tal como foi descrito.



* Para simular, sintetizar ou implementar este módulo, deve primeiro pressionar o botão direito do rato sobre o módulo (na janela *Sources*), e depois escolher a opção **Set as Top** que aparece na janela de diálogo. Esta opção permite definir o módulo como sendo o circuito a considerar nas ações seguintes. 


* Crie um ficheiro de estímulos para este módulo e simule o seu funcionamento. 


* Adicione o sinal interno **n_reg** à janela de ondas do simulador. Coloque o sinal **n_reg** e as saídas **bcd0**, **bcd1**, **bcd2** e **bcd3** no formato decimal. Para isso, pressione com o botão direito do rato sobre o sinal e escolha {Radix}{Unsigned Decimal}. Analise os resultados da simulação.


* Feche a vista *Simulation* na faixa azul.


______________

## Passo 3 - Frequencímetro Digital para baixas frequências ##

### Passo 3.1 ### 

Este circuito pretende medir a frequência de um sinal periódico de entrada. A forma mais simples de implementar um frequencímetro é usar um contador que conte o número de pulsos no sinal de entrada durante um tempo pré-definido (por exemplo, 1 segundo). Apesar de esta solução ser suficiente para um sinal de frequência elevada, não permite medir com precisão sinais de baixa frequência. Neste caso, a melhor forma será medir o seu período e depois inverter, para obter a frequência. É esta a solução que vamos implementar em seguida.


O circuito terá então de realizar as seguintes tarefas:

1. Medição do período do sinal;

2. Cálculo da frequência, fazendo uma operação de divisão (f=1/T);

3. Conversão do resultado (frequência em representação binária) para BCD, para visualização dos visores de 7 segmentos. Esta operação será realizada pelo módulo analisado na secção anterior, definido no ficheiro **bin2bcd.vhd**



Para medir o período do sinal de entrada, vamos usar um contador que conte o número de ciclos de relógio entre dois flancos ascendentes deste sinal. Uma vez que a frequência do sinal de relógio é conhecida (fclk=100MHz), o período do sinal de entrada pode ser facilmente obtido. Por exemplo, se o circuito contar N ciclos, então o período é Tsi=NxTclk e dizemos que a resolução da medição é de Tclk=10ns. Se o relógio deste contador tiver uma frequência inferior a 50MHz, a resolução será necessariamente menor.

O diagrama de fluxo (ASMD) é representado na figura abaixo. Quando a entrada **start** é asserida o circuito fica à espera do primeiro flanco ascendente do sinal de entrada (estado **waite**). Esta ocorrência é assinalada pelo sinal **edge**, gerado por um circuito detector de flanco (discutido no Lab4). Assim que o flanco é detectado, o circuito reinicia os registos **p** e **t** e passa para o estado **count**, onde é realizada a medição temporal. Quando é detectado um segundo flanco o circuito passa para o estado **done**, onde é asserido o sinal **done_tick** que indica o fim da medição. 

São utilizados dois registos. O registo **t** conta entre 0 e 99999 (100000 ciclos de relógio) com um sinal de relógio a 100MHz enquanto o registo **p** só é incrementado quando **t** chega ao fim. Este registo (p) é portanto incrementado com um sinal a 100MHz/100000=1kHz, que resulta numa resolução de 1 milissegundo. Por outro lado este é um registo de 10 bits, logo o seu valor máximo é 1023, o que corresponde a um período máximo de 1023x1ms=1s (ou seja uma frequência mínima de 1Hz). O valor mínimo é 1, o que corresponde a um período máximo de 1ms (ou seja uma frequência máxima de 1kHz).

<div>
<img src="lab5/fig3.png" width="400"/>
</div>


* Adicione o ficheiro **period_counter.vhd** ao seu projeto. Analise o código VHDL. Verifique que o valor da constante **CLK_MS_COUNT** foi alterada para 10 (em vez de 100000) para facilitar a verificação por simulação. 


* Altere esta constante para um GENERIC, de forma a facilitar a sua alteração entre simulação e implementação.


* Adicione agora o ficheiro **slow_freq.vhd** que descreve o circuito representado na figura abaixo - circuito completo de medição de frequência. Ele integra o circuito de medição de período (**period_counter.vhd**), um circuito divisor (**div.vhd**), um conversor binário para BCD (**bin2bcd.vhd**) e uma máquina de estados que controla a sequência de operações. O diagrama ASMD deste controlador é apresentado na figura.


<div>
<img src="lab5/fig4.png" width="900"/>
</div>


* Selecione o ficheiro **slow_freq.vhd** e garanta que é o ficheiro de topo na janela **Sources**. Faça as alterações necessárias para fazer o mapeamento do GENERIC **CLK_MS_COUNT**.


* Crie um ficheiro de estímulos e verifique o funcionamento do circuito. Neste ficheiro: 1) defina o período de relógio como 100us; 2) defina o sinal si como uma onda quadrada com 2000 ciclos de relógio ON e 2000 ciclos OFF. Isto significa que o sinal si tem um período de Tsi=4000xTclk=400ms. Logo a sua frequência é de 2,5Hz. Corra a simulação e verifique se o circuito funciona corretamente. 


* Aumente agora a frequência do sinal si para 400xTclk => fclk=25Hz (Tsi=40ms) e repita a simulação. Note que agora a saída está errada pois a frequência está fora da gama do circuito (que é de 1Hz a 10Hz). Puxe os sinais de entrada e saída do divisor e coloque-os na forma decimal. Verifique que a saída quo é 25641 (25,641Hz), ou seja, o circuito está a medir bem o período, a fazer bem a divisão, mas não mostra bem os resultados pois não mostra os dígitos mais significativos!


* Aumente agora a frequência do sinal si para 40xTclk => fclk=250Hz (Tsi=4ms) e repita a simulação. Note que agora a saída está errada mas o próprio circuito está a medir mal uma vez que o período do sinal a medir está muito próximo da resolução do sistema de medição (1ms).




_________

## Desafio ##

### Desafio 1 ###
Implemente um circuito que permita observar nos visores de 7 segmentos o valor decimal correspondente a um número binário de 8 bits proveniente do conjunto de interruptores. Deve usar o circuito conversor testado no passo 2 deste trabalho, juntamente com o circuito de varrimento dos visores de 7 segmentos, analisado/usado em trabalhos anteriores.

### Desafio 2 ###
Crie um circuito gerador de ondas quadradas que permita testar o funcionamento do frequencímetro na Basys3. A saída do gerador de ondas deve ser encaminhado para um pino de expansão e depois dar entrada na placa noutro pino de expansão, correspondente à entrada do frequencímetro. Lembe-se que para implementação, o GENERIC CLK_MS_COUNT deve ser colocado no valor de 1000000.

